In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from tensorflow.keras.utils import plot_model
import json

In [3]:
import sys 
sys.path.append("../code/")
from model import SNForecastModel

In [4]:
data = np.load("../data/padded_x_train.npy")
data[:,:,3] -= 1 
num_features = data.shape[2]
X_train, y_train = data[:,:-3,:],  data[:,-3:,:]

In [5]:
data_val = np.load("../data/padded_x_val.npy")
data_val[:,:,3] -= 1 

X_val, y_val = data_val[:,:-3,:],  data_val[:,-3:,:]

In [6]:
model = SNForecastModel(units=32, out_steps=3)
losses = {
    "dt_out": "mse",
    "mag_out": "mse",
    "fid_out": "binary_crossentropy"
}
model.compile(optimizer="adam", loss=losses)

In [7]:
x_dt = np.expand_dims(X_train[:,:,0],axis=2)
x_mag = X_train[:,:,1:3]
x_fid = np.expand_dims(X_train[:,:,3],axis=2)
y_dt = np.expand_dims(y_train[:,:,0],axis=2)
y_mag = y_train[:,:,1:3]
y_band = np.expand_dims(y_train[:,:,3],axis=2)
#-----------------------VALIDATION-------------------------------
x_val_dt = np.expand_dims(X_val[:,:,0],axis=2)
x_val_mag = X_val[:,:,1:3]
x_val_fid = np.expand_dims(X_val[:,:,3],axis=2)
y_val_dt = np.expand_dims(y_val[:,:,0],axis=2)
y_val_mag = y_val[:,:,1:3]
y_val_band = np.expand_dims(y_val[:,:,3],axis=2)

In [8]:
dt_min = np.min(x_dt)
dt_max = np.max(x_dt)

x_dt = (x_dt - dt_min)/(dt_max-dt_min)
y_dt = (y_dt - dt_min)/(dt_max-dt_min)
x_val_dt = (x_val_dt - dt_min)/(dt_max-dt_min)
y_val_dt = (y_val_dt - dt_min)/(dt_max-dt_min)

np.savez("../data/dt_norm", {"dt_min": dt_min, "dt_max":dt_max})

In [9]:
mag0_min = np.min(x_mag[:,:,0])
mag0_max = np.max(x_mag[:,:,0])
mag1_min = np.min(x_mag[:,:,0])
mag1_max = np.max(x_mag[:,:,0])

x_mag[:,:,0] = (x_mag[:,:,0] - mag0_min) / (mag0_max-mag0_min)
x_mag[:,:,1] = (x_mag[:,:,1] - mag1_min) / (mag1_max-mag1_min)
y_mag[:,:,0] = (y_mag[:,:,0] - mag0_min) / (mag0_max-mag0_min)
y_mag[:,:,1] = (y_mag[:,:,1] - mag1_min) / (mag1_max-mag1_min)
x_val_mag[:,:,0] = (x_val_mag[:,:,0] - mag0_min) / (mag0_max-mag0_min)
x_val_mag[:,:,1] = (x_val_mag[:,:,1] - mag1_min) / (mag1_max-mag1_min)
y_val_mag[:,:,0] = (y_val_mag[:,:,0] - mag0_min) / (mag0_max-mag0_min)
y_val_mag[:,:,1] = (y_val_mag[:,:,1] - mag1_min) / (mag1_max-mag1_min)


np.savez("../data/mag_norm", {"mag0":{"min":mag0_min, "max":mag0_max},"mag1":{"min":mag1_min, "max":mag1_max}})

In [10]:
ins = {
    "dt_in":x_dt,
    "mag_in": x_mag,
    "fid_in":x_fid
}
outs = {
     "dt_out": y_dt,
    "mag_out": y_mag,
    "fid_out": y_band
}
#-----------------------VALIDATION-------------------------------
ins_val = {
    "dt_in":x_val_dt,
    "mag_in": x_val_mag,
    "fid_in":x_val_fid    
}
outs_val = {
     "dt_out": y_val_dt,
    "mag_out": y_val_mag,
    "fid_out": y_val_band
}

In [11]:
#Early stops
mag_early_stop = tf.keras.callbacks.EarlyStopping( monitor='val_mag_out_loss', min_delta=1e-5, patience=2)
fid_early_stop = tf.keras.callbacks.EarlyStopping( monitor='val_fid_out_loss', min_delta=1e-5, patience=2)
dt_early_stop = tf.keras.callbacks.EarlyStopping( monitor='val_dt_out_loss', min_delta=1e-5, patience=2)

#Tensorboard
tensorboard = tf.keras.callbacks.TensorBoard("../data/training/logs")

#Checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint("../data/training/model_checkpoints/checkpoint", monitor='val_mag_out_loss', verbose=0, save_best_only=True)

callbacks = [mag_early_stop,fid_early_stop,dt_early_stop,tensorboard,checkpoint]

In [12]:
MAX_EPOCHS=100
history = model.fit(ins,outs,
                    batch_size=1000, 
                    epochs=MAX_EPOCHS, 
                    validation_data=(ins_val,outs_val), 
                    callbacks=callbacks)

Epoch 1/100
38/38 [==============================] - 39s 1s/step - loss: 0.8519 - dt_out_loss: 0.0022 - fid_out_loss: 0.6987 - mag_out_loss: 0.1510 - val_loss: 0.6633 - val_dt_out_loss: 7.5895e-04 - val_fid_out_loss: 0.6592 - val_mag_out_loss: 0.0034
Epoch 2/100
38/38 [==============================] - 43s 1s/step - loss: 0.6890 - dt_out_loss: 4.9245e-04 - fid_out_loss: 0.6869 - mag_out_loss: 0.0016 - val_loss: 0.6456 - val_dt_out_loss: 7.6999e-04 - val_fid_out_loss: 0.6439 - val_mag_out_loss: 9.2372e-04
Epoch 3/100
24/38 [=================>............] - ETA: 16s - loss: 0.6877 - dt_out_loss: 3.9466e-04 - fid_out_loss: 0.6863 - mag_out_loss: 9.5393e-04

KeyboardInterrupt: 

In [ ]:
history_dict = history.history
json.dump(history_dict, open("../data/training/history_model.json", 'w'))

In [ ]:
model.save("../data/models/sn_model.tf", save_format="tf")